In [1]:
'''

One imputation method, 10 iterations may take 1 hrs.

test_id 如果是隨機的, 會有引響麼, 理論上只要sort就好, 就算有?
'''
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
import multiprocessing
import warnings
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 
import os
warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(int(time.time()))

NUM_FOLDS = 5
STRATIFIED = True  
TEST_NULL_HYPO = False
ITERATION = (80 if TEST_NULL_HYPO else 10) # It means how many iterations need to get the final stable AUC score.

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified = False):
    '''
    num_folds: int, how many foles u'r going to split.

    Maybe we can write a helper function, to find a best parametres each time when u add a new features, to make sure reliability of experiment.
    But, the experiement time will go up more.
    '''
    #---------------------
    # Divide in training/validation and test data
    #---------------------
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    # 
    logging.info('no bugging in split' if train_df.shape[0] + test_df.shape[0] == df.shape[0] else " opps")
    #---------------------
    # core
    #---------------------
    logging.info("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=int(time.time()))
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=int(time.time()))
    # Create arrays and dataframes to store results
    # train
    oof_preds = np.zeros(train_df.shape[0])
    train_preds = np.zeros(train_df.shape[0])
    # test
    sub_preds = np.zeros(test_df.shape[0])
    # feature importance
    feature_importance_df = pd.DataFrame()

    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    if TEST_NULL_HYPO:
        train_df['TARGET'] = train_df['TARGET'].copy().sample(frac = 1.0).values
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        if TEST_NULL_HYPO:
            clf = LGBMClassifier(
                nthread=int(multiprocessing.cpu_count()*CPU_USE_RATE),
                n_estimators=10000,
                learning_rate=0.02,
                num_leaves=127,
                max_depth=8,
                silent=-1,
                verbose=-1,
                random_state=int(time.time()),
                )
        else:
            clf = LGBMClassifier(
                nthread=int(multiprocessing.cpu_count()*CPU_USE_RATE),
                n_estimators=10000,
                learning_rate=0.02,
                num_leaves=34, # 20
                colsample_bytree=0.2, #0.9497036 < 0.2
                subsample=0.8715623,
                max_depth=8, # 7
                reg_alpha=0.041545473, # 0.3
                reg_lambda=0.0735294,
                min_split_gain=0.0222415,
                min_child_weight=39.3259775, # 60
                silent=-1,
                verbose=-1,
                random_state=int(time.time()),
                )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= False, early_stopping_rounds= 100) # early_stopping_rounds= 200
        # training/validating
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        train_preds[train_idx] += clf.predict_proba(train_x, num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
        # testing
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        logging.info('Fold %2d val AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    logging.info('Over-folds train AUC score : {}'.format(roc_auc_score(train_df['TARGET'], train_preds)))
    
    over_folds_val_auc = roc_auc_score(train_df['TARGET'], oof_preds)
    logging.info('Over-folds val AUC score : {}'.format(over_folds_val_auc))
    
    # # Write submission file and plot feature importance
    # test_df.loc[:,'TARGET'] = sub_preds
    # test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)

    return feature_importance_df, over_folds_val_auc


In [4]:
def main():
    #--------------------
    # load features
    #--------------------
    for i in os.listdir('../features/filled_by_matrix_completion/'):
        df = pd.read_hdf('../features/filled_by_matrix_completion/{}'.format(i))
        logging.info('loading features: {}'.format(i[:-3]))
        #--------------------
        # out-of-fold validating stratigy + LGB
        #--------------------    
        with timer("Run LightGBM with kfold"):
            feature_importance_df = pd.DataFrame()
            over_folds_val_auc_list = np.zeros(ITERATION)
            for i in range(ITERATION):
                logging.info('Iteration %i' %i)
                iter_feat_imp, over_folds_val_auc = kfold_lightgbm(df, num_folds= NUM_FOLDS, stratified= STRATIFIED)
                feature_importance_df = pd.concat([feature_importance_df, iter_feat_imp], axis=0)
                over_folds_val_auc_list[i] = over_folds_val_auc

            logging.info('Over-iterations val AUC score : {}'.format(over_folds_val_auc_list.mean()))
            logging.info('Standard deviation : {}'.format(over_folds_val_auc_list.std()))

            # display_importances(feature_importance_df)
            feature_importance_df_median = feature_importance_df[["feature", "importance"]].groupby("feature").median().sort_values(by="importance", ascending=False)
            useless_features_df = feature_importance_df_median.loc[feature_importance_df_median['importance'] == 0]
            feature_importance_df_mean = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
            #---------------------
            # save
            #---------------------
            output_path = '../output'
            if not os.path.isdir(output_path):
                os.mkdir(output_path)

            if TEST_NULL_HYPO:
                feature_importance_df_mean.to_csv(os.path.join(output_path, 'feature_importance-null_hypo.csv'), index = True)
            else:
                feature_importance_df_mean.to_csv(os.path.join(output_path, 'feature_importance.csv'), index = True)
                useless_features_list = useless_features_df.index.tolist()
                logging.info('useless/overfitting features: \'' + '\', \''.join(useless_features_list) + '\'')


In [5]:
for i in os.listdir('../features/filled_by_matrix_completion/'):
    print (i)
    print (i[:-3])

base_featurs_filled_by_matrix_cluster_2_1.h5
base_featurs_filled_by_matrix_cluster_2_1
base_featurs_filled_by_matrix_cluster_1.h5
base_featurs_filled_by_matrix_cluster_1
base_featurs_filled_by_matrix_cluster_2.h5
base_featurs_filled_by_matrix_cluster_2


In [6]:
CPU_USE_RATE = 0.9
log_dir = '../log_imputating_exp'
init_logging(log_dir)
with timer("Lightgbm run a score"):
    main()


loading features: base_featurs_filled_by_matrix_cluster_2_1
Iteration 0
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.795351
Fold  2 val AUC : 0.793545
Fold  3 val AUC : 0.790812
Fold  4 val AUC : 0.796713
Fold  5 val AUC : 0.795784
Over-folds train AUC score : 0.8845817776244582
Over-folds val AUC score : 0.7944204762363507
Iteration 1
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.792507
Fold  2 val AUC : 0.794181
Fold  3 val AUC : 0.795240
Fold  4 val AUC : 0.790525
Fold  5 val AUC : 0.797802
Over-folds train AUC score : 0.8877447953519423
Over-folds val AUC score : 0.7940287530575084
Iteration 2
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.796648
Fold  2 val AUC : 0.794603
Fold  3 val AUC : 0.797833
Fold  4 val AUC : 0.788744
Fold  5 val AUC : 0.794126
Over-folds train AUC score : 0.8783

Run LightGBM with kfold - done in 4790s


loading features: base_featurs_filled_by_matrix_cluster_1
Iteration 0
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.794788
Fold  2 val AUC : 0.793264
Fold  3 val AUC : 0.805887
Fold  4 val AUC : 0.792813
Fold  5 val AUC : 0.790960
Over-folds train AUC score : 0.8887753136118215
Over-folds val AUC score : 0.7954482818804511
Iteration 1
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.798463
Fold  2 val AUC : 0.797701
Fold  3 val AUC : 0.795757
Fold  4 val AUC : 0.793158
Fold  5 val AUC : 0.791452
Over-folds train AUC score : 0.8884131459180309
Over-folds val AUC score : 0.7953123243407256
Iteration 2
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.793817
Fold  2 val AUC : 0.794547
Fold  3 val AUC : 0.792706
Fold  4 val AUC : 0.803900
Fold  5 val AUC : 0.791768
Over-folds train AUC score : 0.887410

Run LightGBM with kfold - done in 4339s


loading features: base_featurs_filled_by_matrix_cluster_2
Iteration 0
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.789793
Fold  2 val AUC : 0.791475
Fold  3 val AUC : 0.797000
Fold  4 val AUC : 0.800727
Fold  5 val AUC : 0.795061
Over-folds train AUC score : 0.8883388955422977
Over-folds val AUC score : 0.7948267578513687
Iteration 1
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.795619
Fold  2 val AUC : 0.793675
Fold  3 val AUC : 0.793748
Fold  4 val AUC : 0.795617
Fold  5 val AUC : 0.792709
Over-folds train AUC score : 0.8819651692923544
Over-folds val AUC score : 0.7942687485893019
Iteration 2
no bugging in split
Starting LightGBM. Train shape: (307507, 280), test shape: (48744, 280)
Fold  1 val AUC : 0.788190
Fold  2 val AUC : 0.798237
Fold  3 val AUC : 0.794647
Fold  4 val AUC : 0.795678
Fold  5 val AUC : 0.795928
Over-folds train AUC score : 0.881560

Run LightGBM with kfold - done in 3451s
Lightgbm run a score - done in 12584s


In [1]:
# why overefit? 思考一下

# 加一條標準差的虛線...
0.7959288901225524 - 0.0003017394118141831 > 0.795883003439369

False